In [1]:

%load_ext autoreload
%autoreload 2
%matplotlib inline


In [2]:
import tensorflow as tf
import numpy as np
import sys
sys.path.append('../..')
from dev_tools.my_tools import *

Model saved in path: ./log/main.ckpt


In [22]:
tf.concat?

In [57]:
def bb_net(x):
    with tf.name_scope('l1') as scope:
        x_shape = x.get_shape().as_list()
        w = tf.Variable(tf.random_normal([x_shape[1], 5]),name='w')
        b = tf.Variable(tf.random_normal([5]),name='b')
        out = tf.nn.sigmoid(tf.matmul(x, w) + b,name='out')
    with tf.name_scope('l2') as scope:
        w = tf.Variable(tf.random_normal([5, 1]),name='w')
        b = tf.Variable(tf.random_normal([1]),name='b')
        out = tf.nn.sigmoid(tf.matmul(out, w) + b,name='out')
    return out

def get_loss(predict_batches,label_batches,l2_loss):
    predict_batches = tf.reshape(predict_batches,[-1,1])
    label_batches = tf.reshape(label_batches,[-1,1])
    cost = tf.reduce_mean(tf.square(predict_batches - label_batches))
    return cost

def run_model(log_file):
    tf.reset_default_graph()

    input1 = tf.placeholder(tf.float32, [None, 2])
    output1 = bb_net(input1) # 2-20-3 network
#     output1 = bb_net_rename(input1)
    print(output1.shape)


    init_op = tf.global_variables_initializer()
    saver = tf.train.Saver()
    with tf.Session() as sess:
        sess.run(init_op)
    #     sess.run(output1,feed_dict={input1:np.random.rand(3,2)})
        save_path = saver.save(sess, log_file)
        print("Model saved in path: %s" % save_path)
    return

run_model("./log/m1/model_1.ckpt")
run_model("./log/m2/model_2.ckpt")


(?, 1)
Model saved in path: ./log/m1/model_1.ckpt
(?, 1)
Model saved in path: ./log/m2/model_2.ckpt


In [87]:
from tensorflow.python import pywrap_tensorflow 
from tensorflow.python.tools import inspect_checkpoint as chkp

def show_ckpt(filename):
    reader = pywrap_tensorflow.NewCheckpointReader(filename) 
    var_to_shape_map = reader.get_variable_to_shape_map() 
#     print(var_to_shape_map)
    print_sep()
    for key in var_to_shape_map: 
        print("tensor_name: ", key)


    print_sep()
    chkp.print_tensors_in_checkpoint_file(filename, tensor_name='', all_tensors=True)
    
show_ckpt("./log/m1/model_1.ckpt")
show_ckpt("./log/m2/model_2.ckpt")
# show_ckpt('./log/main.ckpt')

---------------------------------------- - ----------------------------------------
tensor_name:  l1/b
tensor_name:  l2/w
tensor_name:  l2/b
tensor_name:  l1/w
---------------------------------------- - ----------------------------------------
tensor_name:  l1/b
[-0.22517745 -0.19252679  1.9404557   1.1618617   0.7309009 ]
tensor_name:  l1/w
[[ 2.1081982  -0.0653644   0.71789426 -0.8337884   0.49091208]
 [-0.79259986 -0.57869947  2.2429605  -0.88159746 -1.350748  ]]
tensor_name:  l2/b
[0.9156851]
tensor_name:  l2/w
[[-0.04598412]
 [ 0.39964706]
 [ 0.7660096 ]
 [ 1.358788  ]
 [-0.19824822]]
---------------------------------------- - ----------------------------------------
tensor_name:  l1/b
tensor_name:  l2/w
tensor_name:  l2/b
tensor_name:  l1/w
---------------------------------------- - ----------------------------------------
tensor_name:  l1/b
[ 1.174079    0.13221252  1.8445915  -1.0480018  -0.8405697 ]
tensor_name:  l1/w
[[-1.3923197  -1.0091283  -0.604495   -1.1805317  -1.468925

In [4]:
def inference(x1,x2):
    with tf.name_scope('net1') as scope:
        with tf.name_scope('l1') as scope:
            x_shape = x1.get_shape().as_list()
            w = tf.Variable(tf.random_normal([x_shape[1], 5]),name='w')
            b = tf.Variable(tf.random_normal([5]),name='b')
        net1_out = tf.nn.sigmoid(tf.matmul(x1, w) + b)
    with tf.name_scope('net2') as scope:
        with tf.name_scope('l1') as scope:
            x_shape = x2.get_shape().as_list()
            w = tf.Variable(tf.random_normal([x_shape[1], 5]),name='w')
            b = tf.Variable(tf.random_normal([5]),name='b')
        net2_out = tf.nn.sigmoid(tf.matmul(x2, w) + b)
    
#     pdb.set_trace()
    with tf.name_scope('final_layer') as scope:
        w = tf.Variable(tf.random_normal([10, 1]),name='w')
        b = tf.Variable(tf.random_normal([1]),name='b')
        final_out = tf.nn.sigmoid(tf.matmul(tf.concat([net1_out,net2_out],1), w) + b)

    return final_out 

def run_main():
    tf.reset_default_graph()

    input1 = tf.placeholder(tf.float32, [None, 2])
    input2 = tf.placeholder(tf.float32, [None, 2])
    output = inference(input1,input2) 

#     init_op = tf.global_variables_initializer()
#     saver = tf.train.Saver()
#     with tf.Session() as sess:
#         sess.run(init_op)
#         save_path = saver.save(sess, './log/main.ckpt')
#         print("Model saved in path: %s" % './log/main.ckpt')
#     return
    
    saver1 = tf.train.Saver({'l1/b':'net1/l1/b'})
    with tf.Session() as sess:
        sess.run(init_op)
        cons_input = np.random.rand(3,2)
        sess.run(output1,feed_dict={input1:cons_input,input2:cons_input})
        save_path = saver.save(sess, log_file)
        print("Model saved in path: %s" % save_path)
    return  

run_main()

In [48]:
for_input = np.random.rand(3,2)

In [110]:
from tensorflow.python.framework import graph_util



def test():
    with tf.Graph().as_default() as g_1: 
        ckpt = tf.train.get_checkpoint_state('./log/m1/') 
        with tf.Session(graph=g_1) as sess: 

            saver = tf.train.import_meta_graph('./log/m1/model_1.ckpt.meta') 
            print(ckpt.model_checkpoint_path)
            saver.restore(sess, ckpt.model_checkpoint_path)
    #         writer = tf.summary.FileWriter("./log", sess.graph)
    #         print(sess.graph.get_all_collection_keys())
    #         print(sess.graph.get_collection('variables'))
    #         print(sess.graph.get_collection('trainable_variables'))
    #         print(sess.graph.get_tensor_by_name('l1/out:0'))
            input = sess.graph.get_tensor_by_name('Placeholder:0')
            output = sess.graph.get_tensor_by_name('l1/out:0')
            print(sess.run(output,{input:for_input}))
    #         print(sess.graph_def)
            g1def = graph_util.convert_variables_to_constants( 
                sess, 
                sess.graph_def, 
    #             ["l1/out",'l2/out'])
                ["l1/out"])

    with tf.Graph().as_default() as g_2: 
        with tf.Session(graph=g_1) as sess: 
            saver = tf.train.import_meta_graph('./log/m2/model_2.ckpt.meta') 
            saver.restore(sess, './log/m2/model_2.ckpt')
            input = sess.graph.get_tensor_by_name('Placeholder:0')
            output = sess.graph.get_tensor_by_name('l1/out:0')
            print(sess.run(output,{input:for_input}))
    #         print(sess.graph_def)
            g2def = graph_util.convert_variables_to_constants( 
                sess, 
                sess.graph_def, 
                ["l1/out"])


    with tf.Graph().as_default() as g_comb: 
        with tf.Session(graph=g_comb) as sess: 
            x1 = tf.placeholder(tf.float32, name="input_1")
            x2 = tf.placeholder(tf.float32, name='input_2')
            out1 = tf.import_graph_def(g1def,input_map={"Placeholder:0": x1}, return_elements=["l1/out:0"])[0]
            out2 = tf.import_graph_def(g2def,input_map={"Placeholder:0": x2}, return_elements=["l1/out:0"])[0]
            print(out1)
            print(out2)
#             pdb.set_trace()
            with tf.name_scope('final_layer') as scope:
                w = tf.Variable(tf.random_normal([10, 1]),name='w')
                b = tf.Variable(tf.random_normal([1]),name='b')
                final_out = tf.nn.sigmoid(tf.matmul(tf.concat([out1,out2],1), w) + b, name='final_out')

            saver = tf.train.Saver()


            writer = tf.summary.FileWriter("./log", sess.graph)
            print(sess.run(out1,{x1:for_input,x2:for_input}))
            print((sess.graph.get_tensor_by_name('import/l1/w:0')).eval())
            print((sess.graph.get_tensor_by_name('import_1/l1/w:0')).eval())
            sess.run(tf.global_variables_initializer())
            
            save_path = saver.save(sess, './log/comb/main.ckpt')
            print("Model saved in path: %s" % './log/comb/main.ckpt')

test()   
show_ckpt('./log/comb/main.ckpt')

./log/m1/model_1.ckpt
INFO:tensorflow:Restoring parameters from ./log/m1/model_1.ckpt
[[0.6998952  0.42184424 0.9357194  0.6366171  0.6916014 ]
 [0.767608   0.43084663 0.9307523  0.62652844 0.7272433 ]
 [0.29320782 0.33458742 0.9794445  0.59869456 0.39708373]]
INFO:tensorflow:Froze 2 variables.
INFO:tensorflow:Converted 2 variables to const ops.
INFO:tensorflow:Restoring parameters from ./log/m2/model_2.ckpt
[[0.5864791  0.41253614 0.83595526 0.12428858 0.17414013]
 [0.5452906  0.36942482 0.81458646 0.12073739 0.13997047]
 [0.7179354  0.6464656  0.92809665 0.08064602 0.448617  ]]
INFO:tensorflow:Froze 2 variables.
INFO:tensorflow:Converted 2 variables to const ops.
Tensor("import/l1/out:0", shape=(?, 5), dtype=float32)
Tensor("import_1/l1/out:0", shape=(?, 5), dtype=float32)
[[0.6998952  0.42184424 0.9357194  0.6366171  0.6916014 ]
 [0.767608   0.43084663 0.9307523  0.62652844 0.7272433 ]
 [0.29320782 0.33458742 0.9794445  0.59869456 0.39708373]]
[[ 2.1081982  -0.0653644   0.71789426 -

In [ ]:
[[ 2.1081982  -0.0653644   0.71789426 -0.8337884   0.49091208]
 [-0.79259986 -0.57869947  2.2429605  -0.88159746 -1.350748  ]]

[[-1.3923197  -1.0091283  -0.604495   -1.1805317  -1.4689256 ]
 [-0.26353896  0.56431895  0.8425082  -1.6075876   0.760832  ]]

In [106]:
!tensorboard --logdir ./log/

W0509 15:37:45.916058 Reloader tf_logging.py:120] Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
W0509 15:37:45.916058 140716453242624 tf_logging.py:120] Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
W0509 15:37:45.923168 Reloader tf_logging.py:120] Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
W0509 15:37:45.923167 140716453242624 tf_logging.py:120] Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
W0509 15:37:45.929723 Reloader tf_logging.py:120] Found more than one graph event per run, or 

W0509 15:37:46.023013 Reloader tf_logging.py:120] Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
W0509 15:37:46.023013 140716453242624 tf_logging.py:120] Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
W0509 15:37:46.025897 Reloader tf_logging.py:120] Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
W0509 15:37:46.025897 140716453242624 tf_logging.py:120] Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
TensorBoard 1.11.0 at http://woody-System-Product-Name:6006 (Press CTRL+C to quit)
W0509 15:37

In [113]:
def test_restore():
    with tf.Graph().as_default() as g_1: 
        ckpt = tf.train.get_checkpoint_state('./log/m1/') 
        with tf.Session(graph=g_1) as sess: 

            saver = tf.train.import_meta_graph('./log/m1/model_1.ckpt.meta') 
            print(ckpt.model_checkpoint_path)
            saver.restore(sess, ckpt.model_checkpoint_path)
            input = sess.graph.get_tensor_by_name('Placeholder:0')
            output = sess.graph.get_tensor_by_name('l1/out:0')
            print(sess.run(output,{input:for_input}))
            g1def = graph_util.convert_variables_to_constants( 
                sess, 
                sess.graph_def, 
                ["l1/out"])

    with tf.Graph().as_default() as g_2: 
        with tf.Session(graph=g_1) as sess: 
            saver = tf.train.import_meta_graph('./log/m2/model_2.ckpt.meta') 
            saver.restore(sess, './log/m2/model_2.ckpt')
            input = sess.graph.get_tensor_by_name('Placeholder:0')
            output = sess.graph.get_tensor_by_name('l1/out:0')
            print(sess.run(output,{input:for_input}))
            g2def = graph_util.convert_variables_to_constants( 
                sess, 
                sess.graph_def, 
                ["l1/out"])


    with tf.Graph().as_default() as g_comb: 
        with tf.Session(graph=g_comb) as sess: 
            x1 = tf.placeholder(tf.float32, name="input_1")
            x2 = tf.placeholder(tf.float32, name='input_2')
            out1 = tf.import_graph_def(g1def,input_map={"Placeholder:0": x1}, return_elements=["l1/out:0"])[0]
            out2 = tf.import_graph_def(g2def,input_map={"Placeholder:0": x2}, return_elements=["l1/out:0"])[0]
#             pdb.set_trace()
            with tf.name_scope('final_layer') as scope:
                w = tf.Variable(tf.random_normal([10, 1]),name='w')
                b = tf.Variable(tf.random_normal([1]),name='b')
                final_out = tf.nn.sigmoid(tf.matmul(tf.concat([out1,out2],1), w) + b, name='final_out')

            saver = tf.train.Saver()


            writer = tf.summary.FileWriter("./log", sess.graph)
            print(sess.run(out1,{x1:for_input,x2:for_input}))
            print((sess.graph.get_tensor_by_name('import/l1/w:0')).eval())
            print((sess.graph.get_tensor_by_name('import_1/l1/w:0')).eval())
            
            save_path = saver.restore(sess, './log/comb/main.ckpt')
            print(w.eval())
test_restore()

./log/m1/model_1.ckpt
INFO:tensorflow:Restoring parameters from ./log/m1/model_1.ckpt
[[0.6998952  0.42184424 0.9357194  0.6366171  0.6916014 ]
 [0.767608   0.43084663 0.9307523  0.62652844 0.7272433 ]
 [0.29320782 0.33458742 0.9794445  0.59869456 0.39708373]]
INFO:tensorflow:Froze 2 variables.
INFO:tensorflow:Converted 2 variables to const ops.
INFO:tensorflow:Restoring parameters from ./log/m2/model_2.ckpt
[[0.5864791  0.41253614 0.83595526 0.12428858 0.17414013]
 [0.5452906  0.36942482 0.81458646 0.12073739 0.13997047]
 [0.7179354  0.6464656  0.92809665 0.08064602 0.448617  ]]
INFO:tensorflow:Froze 2 variables.
INFO:tensorflow:Converted 2 variables to const ops.
[[0.6998952  0.42184424 0.9357194  0.6366171  0.6916014 ]
 [0.767608   0.43084663 0.9307523  0.62652844 0.7272433 ]
 [0.29320782 0.33458742 0.9794445  0.59869456 0.39708373]]
[[ 2.1081982  -0.0653644   0.71789426 -0.8337884   0.49091208]
 [-0.79259986 -0.57869947  2.2429605  -0.88159746 -1.350748  ]]
[[-1.3923197  -1.0091283

In [ ]:
tensor_name:  final_layer/b
[-1.3819424]
tensor_name:  final_layer/w
[[-0.30363435]
 [ 0.34300038]
 [ 1.0742702 ]
 [-0.38556582]
 [-0.1977814 ]
 [-0.4899265 ]
 [ 0.766434  ]
 [ 0.57979876]
 [ 0.8123569 ]
 [-0.6349885 ]]

In [115]:
tf.Graph?

In [28]:
show_ckpt('../nki_alff/log/model_mse_woody.ckpt')

---------------------------------------- - ----------------------------------------
tensor_name:  batch_normalization_4/gamma/Adam
tensor_name:  l2_conv3d/kernel/Adam
tensor_name:  l2_conv3d/b/Adam
tensor_name:  batch_normalization_5/beta/Adam
tensor_name:  batch_normalization/gamma
tensor_name:  l4_conv3d/kernel/Adam_1
tensor_name:  batch_normalization_4/beta/Adam_1
tensor_name:  batch_normalization_2/gamma
tensor_name:  batch_normalization_3/moving_mean
tensor_name:  l1_conv3d/kernel/Adam
tensor_name:  batch_normalization/beta/Adam_1
tensor_name:  l8_fc/b/Adam_1
tensor_name:  batch_normalization/moving_variance
tensor_name:  batch_normalization_6/moving_variance
tensor_name:  batch_normalization/gamma/Adam_1
tensor_name:  l7_fc/w/Adam_1
tensor_name:  batch_normalization_2/beta/Adam_1
tensor_name:  l6_fc/w
tensor_name:  batch_normalization_1/beta
tensor_name:  l5_conv3d/b/Adam
tensor_name:  batch_normalization_3/beta/Adam_1
tensor_name:  beta1_power
tensor_name:  l8_fc/b
tensor_name: 

In [29]:
tf.compat.v1?

Object `tf.compat.v1` not found.


In [33]:
g_1 = tf.Graph()
with g_1.as_default():
  # Operations created in this scope will be added to `g_1`.
  c = tf.constant("Node in g_1")

  # Sessions created in this scope will run operations from `g_1`.
  sess_1 = tf.Session()

g_2 = tf.Graph()
with g_2.as_default():
  # Operations created in this scope will be added to `g_2`.
  d = tf.constant("Node in g_2")

# Alternatively, you can pass a graph when constructing a <a href="./../api_docs/python/tf/Session"><code>tf.Session</code></a>:
# `sess_2` will run operations from `g_2`.
sess_2 = tf.Session(graph=g_2)

assert c.graph is g_1
assert sess_1.graph is g_1

assert d.graph is g_2
assert sess_2.graph is g_2